# 8 school example

自分で Jupyter Notebook を実行したい場合は、
* ツールバーの Cell -> All Output -> Clear
* あとは上のセルから順にツールバーの再生ボタンで実行するか、Cell -> RunAll で全実行

以下、[PyStanのGetting Started](https://pystan.readthedocs.io/en/latest/getting_started.html)のコピーです。

## インストール

* 前提：python3 環境は anaconda とかで各自導入してあること
* 前提：C++ コンパイラが使えること
* PyStan の導入は `$ pip install pystan` でOK

## ライブラリ読み込み
* matplotlibがjupyter内で描画されるように
* pystanのimport

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import pystan

## stan のコード
別ファイルで書いても良いが、文字列で与えてもOK

* data
   + $J$
   + $\{y_j\}^J_{j=1}$
   + $\{\sigma_j\}^J_{j=1}$
* parameters
   + $\mu$
   + $\tau$
   + $\{\eta_j\}^J_{j=1}$
* transformed parameters
   + $\theta_j = \mu + \tau \eta_j \; (j=1 \cdots J)$
* model
   + $\eta_j \sim \mathcal{N}(0, 1)$
   + $y_j \sim \mathcal{N}(\theta_j, \sigma_j)$

In [2]:
schools_code = """
data {
    int<lower=0> J; // number of schools
    real y[J]; // estimated treatment effects
    real<lower=0> sigma[J]; // s.e. of effect estimates
}
parameters {
    real mu;
    real<lower=0> tau;
    real eta[J];
}
transformed parameters {
    real theta[J];
    for (j in 1:J)
    theta[j] = mu + tau * eta[j];
}
model {
    eta ~ normal(0, 1);
    y ~ normal(theta, sigma);
}
"""

## data
* ディクショナリでデータを作る
* key = データの変数名(文字列)
* value = スカラー値、ベクトル値はリスト

In [3]:
schools_dat = {'J': 8,
               'y': [28,  8, -3,  7, -1,  1, 18, 12],
               'sigma': [15, 10, 16, 11,  9, 11, 10, 18]}

## pystan の実行
stanコードの与え方
* `model_code=schools_code` : 上記のように文字列で与える
* `file='8schools.stan'` : stanコードのファイルで与える

In [4]:
%time fit = pystan.stan(model_code=schools_code, data=schools_dat, iter=1000, chains=4)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_cbe9cd2f1e5ab5d1c7cce1f23ca970b4 NOW.


CPU times: user 1.05 s, sys: 104 ms, total: 1.16 s
Wall time: 27.2 s


## 計算結果の取り出し

In [ ]:
la = fit.extract(permuted=True)  # return a dictionary of arrays
mu = la['mu']
print(mu)

## return an array of three dimensions: iterations, chains, parameters
a = fit.extract(permuted=False)

In [ ]:
print(fit)

## グラフの描画

In [ ]:
fig = fit.plot('mu')
fig = fit.plot('tau')
fig = fit.plot('eta')
fig = fit.plot('theta')